In [2]:

!pip install ipdb
import json
import os
import re
import pandas as pd
from google.colab import drive
import nltk
import string
from nltk.tokenize import word_tokenize 
import re
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
from spacy.lang.en import English
import spacy
from collections import Counter
import ipdb
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
%%capture
drive.mount('/content/drive')

In [4]:
path = f'/content/drive/MyDrive/ai_project/dataset_processing/datasets_amazon_cleaned/'
metadata_path = f'/content/drive/MyDrive/ai_project/dataset_processing/cleaned_big_metadata/'

In [5]:
# Define Tokenisation functions

# Run regex commands to parse the 'title' entries into lists without:
# numbers
# html tags
# whitespace
# punctuation

def remove_span_tags(text):
   
    clean = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(clean,'',text)
    
def remove_html_tags(text):
    """Remove html tags from a string"""
    
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
def remove_numbers_with_letters(text): 
    '''  
    This function takes strings containing numbers and returns strings with numbers removed.
    '''
    text = re.sub(r'\d+\w+\d+','',text)
    text = re.sub(r'\d+\w+','',text)
    text = re.sub(r'\d+','',text)
    
    return text    
    #this is may need to changed according to which numbers are useful for classification
    
def remove_whitespace(text): 
    '''
    This function takes strings containing mentions and returns strings with 
    whitespaces removed.
    '''
    
    return " ".join(text.split())
def remove_punctuation(text): 
    """
    This function takes strings containing self defined punctuations and returns
    strings with punctuations removed.
    """
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

def remove_stopwords(tokenized_text): 
    '''
    This function takes a list of tokenized words from the description and title, removes self-defined stop words from the list,
    and returns the list of words with stop words removed
    '''
    try: 
      filtered_text = [word for word in tokenized_text if word not in stopwords_list] 
    except:
      filtered_text = []
    return filtered_text

In [20]:
nltk.download('punkt')
nlp = spacy.load("en_core_web_sm")


def tokenize_sent(text): 
  word_tokens = word_tokenize(text)  
  return word_tokens 


def tokenize_and_lemmatize(series): 
  empty_list = []
  for doc in nlp.pipe(series, batch_size=32, n_process=3, disable=["parser", "ner"]):
    next_item = [tok.lemma_ for tok in doc if not tok.lemma_.isspace()]
    empty_list.append(next_item)

  return empty_list 


def detokenize_sent(text): 
    ''' 
    This function takes a list of words and returns a string.
    '''
    if type(text) == list:
      word_detokens = TreebankWordDetokenizer().detokenize(text)
    else:
      word_detokens = ''


      
    return word_detokens


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### *Pipes to 1 csv file*

In [ ]:
# data_set = pd.DataFrame()

# for file in os.listdir(metadata_path):
#   if file[-4:] == '.csv':
#     print(f'\rProcessing {file}',end='')
#     file_name = file.split('.')[0]
#     df = pd.read_csv(f'{metadata_path}{file}')
#     cont = True
#     """try: 
#       t = df[df['category'].map(bool)].reset_index(drop=True)
#       if len(t) > 25000 and cont: df, cont = t, False
#     except: 
#       print('Had no categories')"""

#     # Gives the dataframe a new attribute "main_cat" using the file name 
#     df['main_cat'] = file_name

#     # Checks there's a valid title: Not NaN or longer than 200 characters
#     try:
#       df = df.dropna(subset=['title'])
#       df = df[df['title'].apply(lambda x : len(x) < 200)]
    
#       # Drops row if the Price column is NaN
#       t = df.dropna(subset=['price'])
#       # Checks remaining dataframe is bigger than 25k rows
#       if len(t) > 25000 and cont: df, cont = t, False
#     except:
#       print(f'\rProblem with {file}')
    
#     data_set = data_set.append(df)
# data_set = data_set.reset_index(drop=True)

#### *Processes each metadata csv into a slimmed 25k csv*

In [29]:
data_set = pd.DataFrame()

output_path = f'/content/drive/MyDrive/ai_project/dataset_processing/datasets_amazon_cleaned/all_metadata_files_upto25k/'

for file in os.listdir(metadata_path):
  if file[-4:] == '.csv':
    print(f'\rProcessing {file}',end='')
    file_name = file.split('.')[0]
    df = pd.read_csv(f'{metadata_path}{file}')

    # Gives the dataframe a new attribute "main_cat" using the file name 
    # try:
    df['main_cat'] = file_name

        # dropping NaNs
    df = df.dropna(axis = 0 )
        # Getting rid of empty imageURLs
    df = df[df['imageURL'].apply(lambda x: len(x) > 3)]
      
        # Getting rid of excessively long titles
    df = df[df['title'].apply(lambda x: len(x) < 250)]

        #getting rid of excessively long prices

    df = df[df['price'].apply(lambda x: len(x) <= 8)]

  
        #cleaning title
    df["cleaned_title"] = df["title"].apply(lambda x: remove_html_tags(str(x)))
    df["cleaned_title"] = df["cleaned_title"].apply(lambda x: str(x).lower())
    df["cleaned_title"] = df["cleaned_title"].apply(lambda x: remove_whitespace(str(x))).apply(lambda x: remove_punctuation(str(x))).apply(lambda x: remove_numbers_with_letters(str(x)))
    df["cleaned_title"] = df["cleaned_title"].apply(lambda x: remove_html_tags(str(x)))
    df["cleaned_title"] = df["cleaned_title"].apply(lambda x: remove_span_tags(str(x)))
      # converting to lowercase
    df["cleaned_title"] = df["cleaned_title"].apply(lambda x: str(x).lower())
       #tokenizing the title
    df["title_token"] = df["cleaned_title"].apply(word_tokenize)
        #searching for stopwords
      
    counter = Counter()
    for word in  [w for sent in df["title_token"] for w in sent]:
       counter[word] += 1        
    counter.most_common(15)
    top_n = 10
    stopwords_list = set([word for (word, count) in counter.most_common(top_n)])
    #removing the stopwords 

    df["title_stopwords"] = df["title_token"].apply(remove_stopwords)
    df["title_stopwords"] = df["title_stopwords"].apply(detokenize_sent)

    thelist = tokenize_and_lemmatize(df['cleaned_title'].to_list())
    df = df.assign(title_lemmatize_tokens = thelist)
    
    df["title_lemmatize"] = df["title_lemmatize_tokens"].apply(detokenize_sent)


    df["title_stopwords_lemmatize_tokens"] = df["title_lemmatize_tokens"].apply(remove_stopwords)
    df["title_stopwords_lemmatize"] = df["title_stopwords_lemmatize_tokens"].apply(detokenize_sent)
   
    

       # dropping any strange titles which didn't format properly
    df = df.dropna()
    df = df.reset_index(drop=True)
        # Removes empty titles after cleaning for stopwords
    df = df.iloc[list(df[~(df["title_stopwords"].apply(lambda x : len(x)) == 0)].index)]
        #reseting the index
        
      # Shuffling the rows 
    df = df.sample(frac = 1)
    data_ready = df

  
    try:
      df = data_ready.sample(25000)
      df = df.reset_index(drop=True)
    except:
      print(f'\r{file} has less than 25000 rows')
    df.reset_index(drop=True).to_csv(f'{output_path}{file}',index=False)

Video_Games.csv has less than 25000 rows
Software.csv has less than 25000 rows
Prime_Pantry.csv has less than 25000 rows
Movies_and_TV.csv has less than 25000 rows
Magazine_Subscriptions.csv has less than 25000 rows
Luxury_Beauty.csv has less than 25000 rows
Gift_Cards.csv has less than 25000 rows
Digital_Music.csv has less than 25000 rows
Books.csv has less than 25000 rows
Applicances.csv has less than 25000 rows
Amazon_Fashion.csv has less than 25000 rows
All_Beauty.csv has less than 25000 rows
